In [1]:
import pandas as pd
from scipy.spatial import distance
import numpy as np

df = pd.read_csv('./data/fireData_mapping.csv', encoding='CP949')
# 관측소 좌표
station = np.array([[127.0607, 37.90188 ],[126.983, 37.25746 ],[127.49446, 37.48863 ],[127.48421, 37.26399 ],[126.76648, 37.88589 ]])
station_name = ['동두천', '수원','양평','이천','파주']

print(df)


        No  Column1 location        stdt        eddt  damagearea          경도  \
0        1        0     경기평택  2019-08-26  2019-08-26        0.01  127.003713   
1        2        1     경기포천  2019-09-29  2019-09-30        0.16  127.237579   
2        3        2     경기평택  2019-10-12  2019-10-12        0.03  127.003713   
3        4        3     경기안양  2019-10-19  2019-10-19        0.01  126.902771   
4        5        4     경기가평  2019-11-07  2019-11-07        0.25  127.407659   
...    ...      ...      ...         ...         ...         ...         ...   
1055  1056     1055    경기남양주  2017-03-18  2017-03-18        0.05  127.258003   
1056  1057     1056    경기남양주  2017-03-18  2017-03-18        0.02  127.258003   
1057  1058     1057     경기양평  2017-03-18  2017-03-18        2.00  127.581096   
1058  1059     1058     경기가평  2017-03-18  2017-03-18        0.15  127.407659   
1059  1060     1059     경기양평  2017-03-19  2017-03-19        1.30  127.581096   

             위도    상태  
0     37.023032

In [2]:
df = df[df['경도']<128]

df_No = df['No']
# 위도 경도를 2중 리스트로 만듬
df_numpy = df.loc[:,'경도':'위도'].values
location = df['location']
df2 = df.set_index("No")
result=[]
# 거리 계산
for i in df_numpy:
    for j in station:
        tmp = distance.euclidean(i, j)
        result.append(tmp)
        
result = np.array(result)
result = np.reshape(result, (len(df_numpy), len(station)))
result = pd.DataFrame(result, columns=station_name, index=df_No)
print(result)

           동두천        수원        양평        이천        파주
No                                                    
1     0.880694  0.235341  0.676472  0.537530  0.894876
2     0.188717  0.754457  0.543567  0.745647  0.478144
3     0.880694  0.235341  0.676472  0.537530  0.894876
4     0.521809  0.167542  0.597634  0.598187  0.500269
5     0.358602  0.697864  0.334089  0.552584  0.645510
...        ...       ...       ...       ...       ...
1056  0.325520  0.473545  0.282369  0.441356  0.548275
1057  0.325520  0.473545  0.282369  0.441356  0.548275
1058  0.646074  0.652778  0.091803  0.272791  0.893426
1059  0.358602  0.697864  0.334089  0.552584  0.645510
1060  0.646074  0.652778  0.091803  0.272791  0.893426

[1046 rows x 5 columns]


In [3]:
# 가장 가까운 관측소 찾기
near = result.loc[:,'동두천':'파주'].min(axis=1)
value = near.values
result = result.loc[:,:].isin(value)
print(result)
# result['관측소'] = value
result_t = result.transpose()
last = pd.concat([df2, result], axis=1)
print(result_t)

        동두천     수원     양평     이천     파주
No                                     
1     False   True  False  False  False
2      True  False  False  False  False
3     False   True  False  False  False
4     False   True  False  False  False
5     False  False   True  False  False
...     ...    ...    ...    ...    ...
1056  False  False   True  False  False
1057  False  False   True  False  False
1058  False  False   True  False  False
1059  False  False   True  False  False
1060  False  False   True  False  False

[1046 rows x 5 columns]
No    1      2      3      4      5      6      7      8      9      10    \
동두천  False   True  False  False  False  False  False  False  False  False   
수원    True  False   True   True  False  False   True  False  False  False   
양평   False  False  False  False   True   True  False   True   True  False   
이천   False  False  False  False  False  False  False  False  False  False   
파주   False  False  False  False  False  False  False  False  False   T

In [4]:
# 가장 짧은 거리의 관측소 이름 가져오기
st=[]
for i in result_t:
    st.append(result_t.index[result_t[i]].tolist()[0])
last['관측소'] = st
print(last)

      Column1 location        stdt        eddt  damagearea          경도  \
No                                                                       
1           0     경기평택  2019-08-26  2019-08-26        0.01  127.003713   
2           1     경기포천  2019-09-29  2019-09-30        0.16  127.237579   
3           2     경기평택  2019-10-12  2019-10-12        0.03  127.003713   
4           3     경기안양  2019-10-19  2019-10-19        0.01  126.902771   
5           4     경기가평  2019-11-07  2019-11-07        0.25  127.407659   
...       ...      ...         ...         ...         ...         ...   
1056     1055    경기남양주  2017-03-18  2017-03-18        0.05  127.258003   
1057     1056    경기남양주  2017-03-18  2017-03-18        0.02  127.258003   
1058     1057     경기양평  2017-03-18  2017-03-18        2.00  127.581096   
1059     1058     경기가평  2017-03-18  2017-03-18        0.15  127.407659   
1060     1059     경기양평  2017-03-19  2017-03-19        1.30  127.581096   

             위도    상태    동두천     수원  

In [5]:
weather = pd.read_csv('./data/weather.csv', encoding='CP949')

all = pd.merge(left = last, right = weather, left_on=['관측소', 'stdt'], right_on=['location','date'],how = 'left')

print(all)
all.to_csv('./data/mapping.csv', encoding = 'utf-8')

      Column1 location_x        stdt        eddt  damagearea          경도  \
0           0       경기평택  2019-08-26  2019-08-26        0.01  127.003713   
1           1       경기포천  2019-09-29  2019-09-30        0.16  127.237579   
2           2       경기평택  2019-10-12  2019-10-12        0.03  127.003713   
3           3       경기안양  2019-10-19  2019-10-19        0.01  126.902771   
4           4       경기가평  2019-11-07  2019-11-07        0.25  127.407659   
...       ...        ...         ...         ...         ...         ...   
1041     1055      경기남양주  2017-03-18  2017-03-18        0.05  127.258003   
1042     1056      경기남양주  2017-03-18  2017-03-18        0.02  127.258003   
1043     1057       경기양평  2017-03-18  2017-03-18        2.00  127.581096   
1044     1058       경기가평  2017-03-18  2017-03-18        0.15  127.407659   
1045     1059       경기양평  2017-03-19  2017-03-19        1.30  127.581096   

             위도    상태    동두천     수원  ...  관측소  Unnamed: 0  location_y  \
0     37.02303